In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

import mne
import pandas
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.datasets import make_classification

from tensorflow.python.client import device_lib


from tensorflow.compat.v1.keras.backend import set_session

config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

config.log_device_placement = True  # to log device placement (on which device the operation ran)

sess = tf.compat.v1.Session(config=config)

set_session(sess)
print(device_lib.list_local_devices())


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10776653102716343587
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1951382279193566074
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 14103434441728631364
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7009469728
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2385854686064116217
physical_device_desc: "device: 0, name: GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capabili

In [2]:
fnames = ["./data/sub-{:02d}/eeg/sub-{:02d}_task-ImaginedEmotion_eeg.set".format(i,i) for i in list(range(1,22)) + list(range(23,36))]
print(fnames)

['./data/sub-01/eeg/sub-01_task-ImaginedEmotion_eeg.set', './data/sub-02/eeg/sub-02_task-ImaginedEmotion_eeg.set', './data/sub-03/eeg/sub-03_task-ImaginedEmotion_eeg.set', './data/sub-04/eeg/sub-04_task-ImaginedEmotion_eeg.set', './data/sub-05/eeg/sub-05_task-ImaginedEmotion_eeg.set', './data/sub-06/eeg/sub-06_task-ImaginedEmotion_eeg.set', './data/sub-07/eeg/sub-07_task-ImaginedEmotion_eeg.set', './data/sub-08/eeg/sub-08_task-ImaginedEmotion_eeg.set', './data/sub-09/eeg/sub-09_task-ImaginedEmotion_eeg.set', './data/sub-10/eeg/sub-10_task-ImaginedEmotion_eeg.set', './data/sub-11/eeg/sub-11_task-ImaginedEmotion_eeg.set', './data/sub-12/eeg/sub-12_task-ImaginedEmotion_eeg.set', './data/sub-13/eeg/sub-13_task-ImaginedEmotion_eeg.set', './data/sub-14/eeg/sub-14_task-ImaginedEmotion_eeg.set', './data/sub-15/eeg/sub-15_task-ImaginedEmotion_eeg.set', './data/sub-16/eeg/sub-16_task-ImaginedEmotion_eeg.set', './data/sub-17/eeg/sub-17_task-ImaginedEmotion_eeg.set', './data/sub-18/eeg/sub-18_task

In [3]:
emotions = ["prebase", "awe", "frustration", "joy", "anger", "happy", "sad", "love", "grief", "compassion", "fear", "content", "jealousy", "relief", "disgust", "excite"]
emotions_dict = {emotions[x]:x for x in range(len(emotions))}
cleaned_files = fnames[0:2] +fnames[3:6] + fnames[7:21] + fnames[23:]
print(cleaned_files)

['./data/sub-01/eeg/sub-01_task-ImaginedEmotion_eeg.set', './data/sub-02/eeg/sub-02_task-ImaginedEmotion_eeg.set', './data/sub-04/eeg/sub-04_task-ImaginedEmotion_eeg.set', './data/sub-05/eeg/sub-05_task-ImaginedEmotion_eeg.set', './data/sub-06/eeg/sub-06_task-ImaginedEmotion_eeg.set', './data/sub-08/eeg/sub-08_task-ImaginedEmotion_eeg.set', './data/sub-09/eeg/sub-09_task-ImaginedEmotion_eeg.set', './data/sub-10/eeg/sub-10_task-ImaginedEmotion_eeg.set', './data/sub-11/eeg/sub-11_task-ImaginedEmotion_eeg.set', './data/sub-12/eeg/sub-12_task-ImaginedEmotion_eeg.set', './data/sub-13/eeg/sub-13_task-ImaginedEmotion_eeg.set', './data/sub-14/eeg/sub-14_task-ImaginedEmotion_eeg.set', './data/sub-15/eeg/sub-15_task-ImaginedEmotion_eeg.set', './data/sub-16/eeg/sub-16_task-ImaginedEmotion_eeg.set', './data/sub-17/eeg/sub-17_task-ImaginedEmotion_eeg.set', './data/sub-18/eeg/sub-18_task-ImaginedEmotion_eeg.set', './data/sub-19/eeg/sub-19_task-ImaginedEmotion_eeg.set', './data/sub-20/eeg/sub-20_task

In [4]:
def get_min_durs(file):
    patient = mne.io.read_raw_eeglab(file)
    events_from_annot, event_dict = mne.events_from_annotations(patient)
    indices = []
    for x in range(len(patient.annotations.description)):
        if patient.annotations.description[x] in emotions:
            indices.append(x)

    num_events = len(events_from_annot)

    for x in range(num_events):
        if x in indices:
            if indices.index(x) < len(indices) - 1:
                y = indices[indices.index(x) + 1]
                events_from_annot[x][1] = (events_from_annot[y - 2][0] - events_from_annot[x][0]) / 256
                events_from_annot[x][0] /= 256
            else:
                print(indices.index(x))
                y = num_events - 1
                events_from_annot[x][1] = (events_from_annot[y - 2][0] - events_from_annot[x][0]) / 256
                events_from_annot[x][0] /= 256

    not_ind = []
    for x in range(num_events):
        if x not in indices:
            not_ind.append(x)

    events_from_annot = np.delete(events_from_annot, not_ind, axis=0)
    onsets = events_from_annot[:, 0] 
    durations = events_from_annot[:, 1] 
    return min(durations)

min_dur = min([get_min_durs(file) for file in cleaned_files])

Reading ./data/sub-01/eeg/sub-01_task-ImaginedEmotion_eeg.fdt
Used Annotations descriptions: ['ExitThankYou', 'FeelingItInstructionsButton', 'ImaginationSuggestions', 'InitialInstructions', 'InstructionsForEnding', 'anger', 'awe', 'compassion', 'content', 'disgust', 'enter', 'excite', 'exit', 'fear', 'frustration', 'grief', 'happy', 'jealousy', 'joy', 'love', 'postbase', 'postbase_instruct', 'prebase', 'prebase_instruct', 'press', 'press1', 'relax', 'relief', 'sad']
15
Reading ./data/sub-02/eeg/sub-02_task-ImaginedEmotion_eeg.fdt
Used Annotations descriptions: ['ExitThankYou', 'FeelingItInstructionsButton', 'ImaginationSuggestions', 'InstructionsForEnding', 'anger', 'awe', 'compassion', 'content', 'disgust', 'enter', 'excite', 'exit', 'fear', 'frustration', 'grief', 'happy', 'jealousy', 'joy', 'love', 'postbase', 'postbase_instruct', 'prebase', 'prebase_instruct', 'press', 'press1', 'relax', 'relief', 'sad']
15
Reading ./data/sub-04/eeg/sub-04_task-ImaginedEmotion_eeg.fdt
Used Annotati

15
Reading ./data/sub-21/eeg/sub-21_task-ImaginedEmotion_eeg.fdt
Used Annotations descriptions: ['ExitThankYou', 'FeelingItInstructionsNoButton', 'ImaginationSuggestions', 'InitialInstructions', 'InstructionsForEnding', 'anger', 'awe', 'compassion', 'content', 'disgust', 'enter', 'excite', 'exit', 'fear', 'frustration', 'grief', 'happy', 'jealousy', 'joy', 'love', 'postbase', 'postbase_instruct', 'prebase', 'prebase_instruct', 'press', 'press1', 'relax', 'relief', 'sad']
15
Reading ./data/sub-25/eeg/sub-25_task-ImaginedEmotion_eeg.fdt
Used Annotations descriptions: ['ExitThankYou', 'FeelingItInstructionsButton', 'ImaginationSuggestions', 'InitialInstructions', 'InstructionsForEnding', 'anger', 'awe', 'compassion', 'content', 'disgust', 'enter', 'excite', 'exit', 'fear', 'frustration', 'grief', 'happy', 'jealousy', 'joy', 'love', 'postbase', 'postbase_instruct', 'prebase', 'prebase_instruct', 'press', 'press1', 'relax', 'relief', 'sad']
15
Reading ./data/sub-26/eeg/sub-26_task-ImaginedE

In [5]:
def get_channels(file):
    patient = mne.io.read_raw_eeglab(file)
    events_from_annot, event_dict = mne.events_from_annotations(patient)
    indices = []
    for x in range(len(patient.annotations.description)):
        if patient.annotations.description[x] in emotions:
            indices.append(x)

    num_events = len(events_from_annot)

    for x in range(num_events):
        if x in indices:
            if indices.index(x) < len(indices) - 1:
                y = indices[indices.index(x) + 1]
                events_from_annot[x][1] = (events_from_annot[y - 2][0] - events_from_annot[x][0]) / 256
                events_from_annot[x][0] /= 256
            else:
                print(indices.index(x))
                y = num_events - 1
                events_from_annot[x][1] = (events_from_annot[y - 2][0] - events_from_annot[x][0]) / 256
                events_from_annot[x][0] /= 256

    not_ind = []
    for x in range(num_events):
        if x not in indices:
            not_ind.append(x)

    events_from_annot = np.delete(events_from_annot, not_ind, axis=0)
    onsets = events_from_annot[:, 0] 
    durations = events_from_annot[:, 1] 
    mapping = {event_dict[x]: x for x in event_dict.keys()}
    descriptions = [mapping[event_id] for event_id in events_from_annot[:, 2]]
    annot_from_events = mne.Annotations(onset=onsets, duration=durations,
                                    description=descriptions,
                                    orig_time=patient.info['meas_date'])
    patient.set_annotations(annot_from_events)
    return patient.ch_names

channels = [set(get_channels(file)) for file in cleaned_files]
intersect = channels[0]

for channel in channels:
    intersect.intersection_update(channel)

intersect = list(intersect)

Reading ./data/sub-01/eeg/sub-01_task-ImaginedEmotion_eeg.fdt
Used Annotations descriptions: ['ExitThankYou', 'FeelingItInstructionsButton', 'ImaginationSuggestions', 'InitialInstructions', 'InstructionsForEnding', 'anger', 'awe', 'compassion', 'content', 'disgust', 'enter', 'excite', 'exit', 'fear', 'frustration', 'grief', 'happy', 'jealousy', 'joy', 'love', 'postbase', 'postbase_instruct', 'prebase', 'prebase_instruct', 'press', 'press1', 'relax', 'relief', 'sad']
15
Reading ./data/sub-02/eeg/sub-02_task-ImaginedEmotion_eeg.fdt
Used Annotations descriptions: ['ExitThankYou', 'FeelingItInstructionsButton', 'ImaginationSuggestions', 'InstructionsForEnding', 'anger', 'awe', 'compassion', 'content', 'disgust', 'enter', 'excite', 'exit', 'fear', 'frustration', 'grief', 'happy', 'jealousy', 'joy', 'love', 'postbase', 'postbase_instruct', 'prebase', 'prebase_instruct', 'press', 'press1', 'relax', 'relief', 'sad']
15
Reading ./data/sub-04/eeg/sub-04_task-ImaginedEmotion_eeg.fdt
Used Annotati

15
Reading ./data/sub-21/eeg/sub-21_task-ImaginedEmotion_eeg.fdt
Used Annotations descriptions: ['ExitThankYou', 'FeelingItInstructionsNoButton', 'ImaginationSuggestions', 'InitialInstructions', 'InstructionsForEnding', 'anger', 'awe', 'compassion', 'content', 'disgust', 'enter', 'excite', 'exit', 'fear', 'frustration', 'grief', 'happy', 'jealousy', 'joy', 'love', 'postbase', 'postbase_instruct', 'prebase', 'prebase_instruct', 'press', 'press1', 'relax', 'relief', 'sad']
15
Reading ./data/sub-25/eeg/sub-25_task-ImaginedEmotion_eeg.fdt
Used Annotations descriptions: ['ExitThankYou', 'FeelingItInstructionsButton', 'ImaginationSuggestions', 'InitialInstructions', 'InstructionsForEnding', 'anger', 'awe', 'compassion', 'content', 'disgust', 'enter', 'excite', 'exit', 'fear', 'frustration', 'grief', 'happy', 'jealousy', 'joy', 'love', 'postbase', 'postbase_instruct', 'prebase', 'prebase_instruct', 'press', 'press1', 'relax', 'relief', 'sad']
15
Reading ./data/sub-26/eeg/sub-26_task-ImaginedE

In [6]:
print(min_dur)
print(intersect)

68
['B6', 'B3', 'F6', 'B24', 'F16', 'F7', 'B29', 'C14', 'B18', 'B11', 'B12', 'C20', 'F26', 'F21', 'B8', 'F17', 'H3', 'B22', 'A10', 'F8', 'A4']


In [7]:
def get_data(file, epoch_duration, channel_list):
    patient = mne.io.read_raw_eeglab(file)
    events_from_annot, event_dict = mne.events_from_annotations(patient)
    indices = []
    for x in range(len(patient.annotations.description)):
        if patient.annotations.description[x] in emotions:
            indices.append(x)

    num_events = len(events_from_annot)

    for x in range(num_events):
        if x in indices:
            if indices.index(x) < len(indices) - 1:
                y = indices[indices.index(x) + 1]
                events_from_annot[x][1] = (events_from_annot[y - 2][0] - events_from_annot[x][0]) / 256
                events_from_annot[x][0] /= 256
            else:
                print(indices.index(x))
                y = num_events - 1
                events_from_annot[x][1] = (events_from_annot[y - 2][0] - events_from_annot[x][0]) / 256
                events_from_annot[x][0] /= 256

    not_ind = []
    for x in range(num_events):
        if x not in indices:
            not_ind.append(x)

    events_from_annot = np.delete(events_from_annot, not_ind, axis=0)
    onsets = events_from_annot[:, 0] 
    durations = events_from_annot[:, 1] 
    mapping = {event_dict[x]: x for x in event_dict.keys()}
    descriptions = [mapping[event_id] for event_id in events_from_annot[:, 2]]
    annot_from_events = mne.Annotations(onset=onsets, duration=durations,
                                    description=descriptions,
                                    orig_time=patient.info['meas_date'])
    patient.set_annotations(annot_from_events)
    descrip = patient.annotations.description
    events_from_annot, event_dict = mne.events_from_annotations(patient)
    epochs = mne.Epochs(patient, events_from_annot, event_dict, picks=channel_list, tmin=0, tmax=epoch_duration, baseline=(None,None), on_missing="warning", preload=True, reject_by_annotation=False)
    del patient
    #return epochs
    data = epochs.get_data()
    del epochs
    return data, descrip


In [8]:
# patient1 = get_epochs(fnames[0])
# patient1.plot_sensors(show_names=True)
# patient2 = get_epochs(fnames[1])
# patient2.plot_sensors(show_names=True)

In [ ]:
data = []
descrip = []
i = 0
for file in cleaned_files:
    i += 1 
    print(file, i)
    data_file, descrip_file = get_data(file, min_dur, intersect)
    print(data_file.shape, descrip_file)
    for x in range(data_file.shape[0]):
        data.append(data_file[x])
        descrip.append(descrip_file[x])
        
data = np.asarray(data)
data.shape

./data/sub-01/eeg/sub-01_task-ImaginedEmotion_eeg.set 1
Reading ./data/sub-01/eeg/sub-01_task-ImaginedEmotion_eeg.fdt
Used Annotations descriptions: ['ExitThankYou', 'FeelingItInstructionsButton', 'ImaginationSuggestions', 'InitialInstructions', 'InstructionsForEnding', 'anger', 'awe', 'compassion', 'content', 'disgust', 'enter', 'excite', 'exit', 'fear', 'frustration', 'grief', 'happy', 'jealousy', 'joy', 'love', 'postbase', 'postbase_instruct', 'prebase', 'prebase_instruct', 'press', 'press1', 'relax', 'relief', 'sad']
15
Used Annotations descriptions: ['anger', 'awe', 'compassion', 'content', 'disgust', 'excite', 'fear', 'frustration', 'grief', 'happy', 'jealousy', 'joy', 'love', 'prebase', 'relief', 'sad']
Not setting metadata
Not setting metadata
16 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 16 events and 17409 original time points ...
0 bad epochs dropped
(16, 21, 17409) ['prebase' 'awe' 'frustration' 'joy' 'anger

15
Used Annotations descriptions: ['anger', 'awe', 'compassion', 'content', 'disgust', 'excite', 'fear', 'frustration', 'grief', 'happy', 'jealousy', 'joy', 'love', 'prebase', 'relief', 'sad']
Not setting metadata
Not setting metadata
16 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 16 events and 17409 original time points ...
0 bad epochs dropped
(16, 21, 17409) ['prebase' 'happy' 'fear' 'awe' 'disgust' 'relief' 'jealousy' 'excite'
 'frustration' 'content' 'anger' 'love' 'grief' 'joy' 'sad' 'compassion']
./data/sub-11/eeg/sub-11_task-ImaginedEmotion_eeg.set 9
Reading ./data/sub-11/eeg/sub-11_task-ImaginedEmotion_eeg.fdt
Used Annotations descriptions: ['ExitThankYou', 'FeelingItInstructionsButton', 'ImaginationSuggestions', 'InitialInstructions', 'InstructionsForEnding', 'anger', 'awe', 'compassion', 'content', 'disgust', 'enter', 'excite', 'exit', 'fear', 'frustration', 'grief', 'happy', 'jealousy', 'joy', 'love', 'postba

In [ ]:
data = np.asarray(data)
print(data.shape)
labels = []
for x in np.asarray([emotions_dict[x] for x in descrip]):
    ar = np.zeros(16)
    ar[x] = 1
    labels.append(ar)
    
labels = np.asarray(labels)

print(labels)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Lambda
from tensorflow.keras.layers import DepthwiseConv2D, BatchNormalization
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.layers import SeparableConv1D 
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

K.set_image_data_format('channels_first')



def TA_CSPNN(nb_classes, Channels = 64, Timesamples = 90, 
             dropOut = 0.25,  timeKernelLen = 100, Ft = 22, Fs = 6):
    
    # Input shape is (trials, 1, number of channels, number of time samples)

    input_e      = Input(shape = (1, Channels, Timesamples))    
    convL1       = Conv2D(Ft, (1, timeKernelLen), padding = 'same',input_shape = (1, Channels, Timesamples), use_bias = False)(input_e)

    bNorm1       = BatchNormalization(axis = 1)(convL1)

    convL2       = DepthwiseConv2D((Channels, 1), use_bias = False, 
                                   depth_multiplier = Fs, depthwise_constraint = max_norm(1.))(bNorm1)  
    bNorm2       = BatchNormalization(axis = 1)(convL2)
    
    lambdaL      = Lambda(lambda x:x**2)(bNorm2)
    aPool        = AveragePooling2D((1, Timesamples))(lambdaL)

    dOutL       = Dropout(dropOut)(aPool)
        
    flatten      = Flatten(name = 'flatten')(dOutL)

    dense        = Dense(nb_classes, name = 'dense')(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input_e, outputs=softmax)


def onedcnn(nb_classes, Channels, Timesamples):
    
    # Input shape is (trials, 1, number of channels, number of time samples)

    input_e      = Input(shape = (Channels, Timesamples))    
    convL1       = SeparableConv1D(32, 256, padding = 'same',activation="relu", input_shape = (Channels,Timesamples), data_format="channels_first")(input_e)
    pool         = tf.keras.layers.MaxPooling1D(60, data_format="channels_first")(convL1)

    flatten      = Flatten(name = 'flatten')(pool)

    dense        = Dense(nb_classes, name = 'dense')(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input_e, outputs=softmax)


In [ ]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU

def rnn(nb_classes, Timesamples, Channels):

    input_e = Input(shape = (Timesamples, Channels))
    gru     = tf.keras.layers.GRU(100)(input_e)
    dense   = Dense(nb_classes, activation="softmax")(gru)
    softmax = Activation('softmax', name = 'softmax')(dense)
    return Model(inputs = input_e, outpus=softmax)


In [ ]:
def conv1dp2(num_classes, input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same", data_format='channels_first')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same", data_format='channels_first')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same", data_format='channels_first')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


In [ ]:
data2 = np.transpose(data, axes=(0,2,1))

data = data2
shape = data.shape
cnn = onedcnn(16, Channels = shape[1], Timesamples = shape[2])
cnn.summary()


cnn2 = conv1dp2(16, shape[1:])
cnn2.summary()

print(shape)
rnn = rnn(16, shape[1], shape[2])
rnn.summary()

In [ ]:
cnn2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
X_train = data[:300]
X_test = data[300:]
y_train = labels[:300]
y_test = labels[300:]
print(X_train.shape)
print(y_train.shape)


# fit the keras model on the dataset
cnn2.fit(X_train, y_train, epochs=15, batch_size=10)


In [ ]:
# evaluate the keras model
_, accuracy = cnn2.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))